In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import hazm

In [ ]:
# Load the saved model and tokenizer
output_dir = "/saved_model"
loaded_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(output_dir)


In [ ]:
# Set the device to CPU
device = torch.device("cpu")

# Load the test data from your DataFrame
# test_data = pd.DataFrame({
#     "text": ["Text 1", "Text 2", "Text 3"],
#     "label": [0, 1, 0]  # Example labels, replace with your actual labels
# })

In [ ]:
# Preprocess the test data
normalized_texts = [hazm.Normalizer().normalize(text) for text in test_data["text"]]
tokenized_texts = [hazm.word_tokenize(text) for text in normalized_texts]

# Encode the tokenized texts using the loaded tokenizer
inputs = loaded_tokenizer.batch_encode_plus(
    tokenized_texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [ ]:
# Move the inputs to the appropriate device
inputs = {k: v.to(device) for k, v in inputs.items()}

In [ ]:
# Make predictions
with torch.no_grad():
    outputs = loaded_model(**inputs)

# Get the predicted labels
predicted_labels = outputs.logits.argmax(dim=1)

In [ ]:
# Convert predicted labels and true labels to lists
predicted_labels = predicted_labels.tolist()
true_labels = test_data["label"].tolist()

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average="weighted")
cm = confusion_matrix(true_labels, predicted_labels)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(cm)